# This notebook further optimize the simulation efficiency by vectorizing the measurement generation & gradient calc and update. Only ConsensusEKF is still performed via a subloop due to its stateful nature, where vectorization is difficult.

**Also, we only consider fixed-topology networks in the experiments below.**

In [4]:

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pickle as pkl
import networkx as nx
from matplotlib import style
import jax.numpy as jnp
from jax import jacfwd, jit,grad
from functools import partial


from virtual_sensor import virtual_sensor
from utils.DynamicFilters import getDynamicFilter,joint_meas_func
from utils.dLdp import analytic_FIM,analytic_dLdp,analytic_dhdz,analytic_dhdq
from utils.ConsensusEKF import ConsensusEKF


%load_ext autoreload
%autoreload 2

## Set up F_tilde data structure

In [5]:
def circulant(i,q,p,prev,post,undirected=False):
    """
        Generate a circulant graph with len(p) nodes, node i connected with [i-prev:i+post],i-prev and i+post included but self-loop eliminated.
    """
    n = len(p)
    G = nx.DiGraph()
    edges = [(j%n,i) for i in range(n) for j in range(i-prev,i+post+1)]
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    if undirected:
        G = G.to_undirected()
    return G
    

In [159]:
N_sen = 6 # Number of sensors.

p_0 = np.random.rand(N_sen,2)*3
qhat_0 = np.random.rand(N_sen,2)*10
q = np.array([6,6])

comm_network_generator=lambda i,q,p:circulant(i,q,p,prev=1,post=0,undirected=True)

N_iter=10
C_gain=0.1
coordinate=False

# Set up virtual sensors
C1=-0.3 # Setting C1 as a negative number mitigates the blowing-up effect when the sensors are close to the source.
C0=0
k=1
b=-2
noise_std = 0.01
minimum_sensing_reading=1e-5

# Data containers
p = np.array(p_0) # Sensor Positins
qhat = np.array(qhat_0)



# Create the list of single-term partial FIM's.
def F_single(dh,qhat,ps):
    A = dh(qhat,ps)
    return A.T.dot(A)

def joint_F_single(dh,qhat,ps): # Verified to be correct.
    # The vectorized version of F_single.
    # The output shape is (N_sensor, q_dim, q_dim).
    # Where output[i]=F_single(dh,qhat,ps[i])
    A = dh(qhat,ps)
    return A[:,np.newaxis,:]*A[:,:,np.newaxis]


# Create the list local estimate of global FIM.
dh = partial(analytic_dhdq,C1s=C1,C0s=C0,ks=k,bs=b)
# F = F_single(dh,q_0[i],ps[i:i+1]) for i in range(n_sen)])
F = joint_F_single(dh,qhat,p)
F_est = F+1e-8*np.eye(2) # Adding a small I to ensure invertibility

G = comm_network_generator(0,q,p)
estimators = [ConsensusEKF(q_0,C_gain=C_gain) for q_0 in qhat_0]

# The initialization of local measurement functions and the derivative functions. 
# This is not very pretty. But is required by Consensus EKF.
hs = []
dhdzs = []
dhdqs = []
C1s=C1*np.ones(N_sen)
C0s = C0*np.ones(N_sen)
ks = k * np.ones(N_sen)
bs = b*np.ones(N_sen)
for i in G.nodes():  
    N_i = [i]+list(G[i])     
    C1s_i=C1s[N_i]
    C0s_i = C0s[N_i]
    ks_i = ks[N_i]
    bs_i = bs[N_i]
    hs.append(partial(joint_meas_func,C1s_i,C0s_i,ks_i,bs_i))# Freeze the coefficients, the signature becomes h(z,ps))
    dhdzs.append(partial(analytic_dhdz,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))
    dhdqs.append(partial(analytic_dhdq,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))



        
   
    # Enter main loop
for _ in range(N_iter):
    # Measure
    r = np.linalg.norm(q-p,axis=1)
    y = k* ((r-C1)**b)+C0 + np.random.randn(N_sen)*noise_std
    y[y<=0]=minimum_sensing_reading # We don't want y to be zero or negative.
    
    
    # Estimate
    zhats = np.array([est.z for est in estimators])
    for i in G.nodes():
        N_i = [i]+list(G[i]) 
        # Simulate query of information from other robots. 
        
        # Estimate
         qhat[i,:]=estimators[i].update_and_estimate_loc(hs[i],dhdzs[i],y[N_i],p[N_i],zhats[N_i])
    
    # Partial FIM Calculation
    F = joint_F_single(dh,qhat,p)

    # FIM Consensus

    # Gradient update
    pass

[[6.58824314 3.30971523]
 [5.22330647 8.49790505]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.35189974 9.37898691]] 5
[[6.58824314 3.30971523]
 [5.22330647 8.49790505]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.64518215 8.05382482]]
[[6.58824314 3.30971523]
 [5.22330647 8.49790505]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.64518215 8.05382482]] 0
[[6.02832    4.43560953]
 [5.22330647 8.49790505]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.64518215 8.05382482]]
[[6.02832    4.43560953]
 [5.22330647 8.49790505]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.64518215 8.05382482]] 1
[[6.02832    4.43560953]
 [5.70266001 7.94210721]
 [8.65244481 8.12928523]
 [7.60654628 7.16345776]
 [1.04833831 2.19751404]
 [2.64518215 8.05382482]]
[[6.02832    4.43560953]
 [5.70266001 7.94210721]
 [8.65244481 8.12928523]
 [7.60654628 